All-in-One rearing analyzer


 by max vaglica
https://github.com/Guillermo-Hidalgo-Gadea/UQOAB/blob/main/First%20DLC%20project/.ipynb_checkpoints/partII-checkpoint.ipynb



Run and restart runtime when prompted
 ,make sure runtime type is 'GPU'



# INITIALIZE DEEPLABCUT SESSION

Next three code blocks MUST be run everytime no matter whether you are training a new network, scoring videos with a trained network, or extracting new outlier frames.

In [ ]:
!pip install statsmodels==0.13.1
!pip install deeplabcut

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tables==3.5.1
!pip install tensorboard==2.7.0
!pip install tensorflow==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.3 MB 7.0 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.7.0
    Uninstalling tables-3.7.0:
      Successfully uninstalled tables-3.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deeplabcut 2.2.1.1 requires tables>=3.7.0, but you have tables 3.5.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 8.3 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

After restarting the runtime, Import all and mount the memlab@iu.edu 
google drive. You may need to close and reopen your browser to login. Copy the code after allowing access to the google drive & paste it into the box.

In [ ]:
%tensorflow_version 1.x
import os
os.environ["DLClight"]="True"
import deeplabcut
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

TensorFlow 1.x selected.
Loading DLC 2.2.1.1...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)
Mounted at /content/drive


In [ ]:
ProjectFolderName = 'all_combined_tracker-aes-2022-06-22'
VideoType = 'mp4' 
#videofile_path = ['/content/drive/My Drive/Projects/DeepLabCut/'+ProjectFolderName+'/videos/']
path_config_file = '/content/drive/My Drive/Projects/DeepLabCut/'+ProjectFolderName+'/config.yaml'
testVideo_path = ['/content/drive/My Drive/Projects/DeepLabCut/'+ProjectFolderName+'/videos/']
!ls '/content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/config.yaml'

'/content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/config.yaml'


# TRAIN A NEW NETWORK

Be sure that the variables above match the path of your training videos

The following code will create the training dataset for the network.

"After running this script the training dataset is created and saved in the project directory under the subdirectory 'training-datasets'

This function also creates new subdirectories under dlc-models and appends the project config.yaml file with the correct path to the training and testing pose configuration file. **These files hold the parameters for training the network.**" 

**The pose_cfg.yaml file will store all configuration for training, edit these parameters to change training settings

In [ ]:
deeplabcut.convertcsv2h5(path_config_file, userfeedback=False)

Attention: /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/labeled-data/G:\My Drive\Projects\DeepLabCut\all_combined_tracker-aes-2022-06-22\videos\MS opto 22 07.30.2021 retrain (1 of 2) does not appear to have labeled data!
Attention: /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/labeled-data/G:\My Drive\Projects\DeepLabCut\all_combined_tracker-aes-2022-06-22\videos\MS opto 23 07.28.2021 retrain (1 of 2) does not appear to have labeled data!
Attention: /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/labeled-data/G:\My Drive\Projects\DeepLabCut\all_combined_tracker-aes-2022-06-22\videos\MS opto 24 07.28.2021 retrain (1 of 2) does not appear to have labeled data!
Attention: /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/labeled-data/G:\My Drive\Projects\DeepLabCut\all_combined_tracker-aes-2022-06-22\videos\MsOpto 23 03.11.2021 (1of2) does not appear to have l

In [ ]:
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([808, 908, 223, ..., 990, 572, 841]),
   array([ 796,  460,  668,  801,  539, 1156,  508, 1096, 1076,  944,  219,
          1211,  516,  404,  911, 1222,  855, 1073, 1094,  635,  900,  817,
           575,  311,  741,  121,  458,  229,  674,  950,  494,  651, 1128,
           876,  557,  755, 1207,  295,  444,  743, 1116,  669,  880,  497,
          1198,  964, 1126,  238,  248, 1075,  818, 1091, 1228, 1159, 1135,
           789, 1018,  429, 1077, 1021,  305,  565])))]

Below begins actual training on the network. You want at least 400,000 iterations. saveiters will save a network every 4500 iterations, this can be changed to any amount. Trained networks are saved as 3 'snapshot' files in the same directory. Change the initial learning to a previous snapshot to continue training from save, in the same format I have done in the yaml file. A checkpoint file is updated and saved for the entire training. All 4 of these files are needed to evaluate the network and analyze new videos.

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1,displayiters=100,saveiters=2000)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3', 'objectA'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'clahe': True,
 'claheratio': 0.1,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_all_combined_trackerJun22/all_combined_tracker_jic95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'edge': False,
 'emboss': {'alpha': [0.0, 1.0], 'embossratio': 0.1, 'strength': [0.5, 1.5]},
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'histeq': True,
 'histeqratio': 0.1,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'lr_init': 

Selecting single-animal trainer
Batch Size is 1
Loading ImageNet-pretrained resnet_50
Display_iters overwritten as 100
Save_iters overwritten as 2000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/dlc-models/iteration-0/all_combined_trackerJun22-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfie

iteration: 100 loss: 0.0564 lr: 0.005
iteration: 200 loss: 0.0194 lr: 0.005
iteration: 300 loss: 0.0202 lr: 0.005
iteration: 400 loss: 0.0181 lr: 0.005
iteration: 500 loss: 0.0177 lr: 0.005
iteration: 600 loss: 0.0180 lr: 0.005
iteration: 700 loss: 0.0167 lr: 0.005
iteration: 800 loss: 0.0153 lr: 0.005
iteration: 900 loss: 0.0156 lr: 0.005
iteration: 1000 loss: 0.0146 lr: 0.005
iteration: 1100 loss: 0.0146 lr: 0.005
iteration: 1200 loss: 0.0130 lr: 0.005
iteration: 1300 loss: 0.0141 lr: 0.005
iteration: 1400 loss: 0.0147 lr: 0.005
iteration: 1500 loss: 0.0131 lr: 0.005
iteration: 1600 loss: 0.0121 lr: 0.005
iteration: 1700 loss: 0.0129 lr: 0.005
iteration: 1800 loss: 0.0123 lr: 0.005
iteration: 1900 loss: 0.0120 lr: 0.005
iteration: 2000 loss: 0.0119 lr: 0.005
iteration: 2100 loss: 0.0114 lr: 0.005
iteration: 2200 loss: 0.0114 lr: 0.005
iteration: 2300 loss: 0.0106 lr: 0.005
iteration: 2400 loss: 0.0111 lr: 0.005
iteration: 2500 loss: 0.0110 lr: 0.005
iteration: 2600 loss: 0.0103 lr: 0

# EVALUATE A TRAINED NETWORK

This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .csv file in a subdirectory under evaluation-results

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=False)

Running  DLC_resnet50_all_combined_trackerJun22shuffle1_292000  with # of training iterations: 292000
Running evaluation ...


1239it [19:40,  1.05it/s]

Analysis is done and the results are stored (see evaluation-results) for snapshot:  snapshot-292000
Results for 292000  training iterations: 95 1 train error: 2.29 pixels. Test error: 4.6  pixels.
With pcutoff of 0.6  train error: 2.29 pixels. Test error: 4.6 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
Please check the results, then choose the best model (snapshot) for prediction. You can update the config.yaml file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise, consider adding more labeled-data and retraining the network (see DeepLabCut workflow Fig 2, Nath 2019)


In [ ]:
deeplabcut.analyze_videos(path_config_file,testVideo_path, videotype='mp4', save_as_csv=True)
#deeplabcut.plot_trajectories(path_config_file,testVideo_path, videotype='mp4')

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3', 'objectA'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_all_combined_trackerJun22/all_combined_tracker_jic95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 4,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 'shuffle': True,

Using snapshot-292000 for model /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/dlc-models/iteration-0/all_combined_trackerJun22-trainset95shuffle1
Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos/RC4_220209_1of2.mp4
Loading  /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos/RC4_220209_1of2.mp4
Duration of video [s]:  277.91 , recorded with  29.98 fps!
Overall # of frames:  8331  found with (before cropping) frame dimensions:  1920 1080
Starting to extract posture


100%|█████████▉| 8300/8331 [12:48<00:02, 10.80it/s]


Saving results in /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos...
Saving csv poses!
Starting to analyze %  /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos/RC6_220217_1of2.mp4
Loading  /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos/RC6_220217_1of2.mp4
Duration of video [s]:  323.11 , recorded with  29.98 fps!
Overall # of frames:  9686  found with (before cropping) frame dimensions:  1920 1080
Starting to extract posture


 99%|█████████▉| 9600/9686 [14:52<00:07, 10.76it/s]


...

100%|█████████▉| 2727/2730 [04:07<00:00, 11.00it/s]


Saving results in /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos...
Saving csv poses!
Starting to analyze %  /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos/Rear2 06.28.2021 (1of2).mp4
Loading  /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos/Rear2 06.28.2021 (1of2).mp4
Duration of video [s]:  86.67 , recorded with  29.98 fps!
Overall # of frames:  2598  found with (before cropping) frame dimensions:  1920 1080
Starting to extract posture


 99%|█████████▉| 2575/2598 [03:56<00:02, 10.88it/s]

Saving results in /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_all_combined_trackerJun22shuffle1_292000'

In [ ]:
deeplabcut.create_labeled_video(path_config_file,testVideo_path, videotype='mp4',draw_skeleton=False,save_frames=False)

To store the coordinates for each body part in a csv file, run the store_coordinates(video_name) function. csv will be located with rest of file.

# REFINE A TRAINED NETWORK


In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,testVideo_path,videotype='mp4', automatic=True)

Analyzing all the videos in the directory...


/usr/local/lib/python3.7/dist-packages/deeplabcut/refine_training_dataset/outlier_frames.py:390: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  sum_ = temp_dt.sum(axis=1, level=1)


Method  jump  found  805  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the parameters (start, stop, p_bound, comparisonbodyparts) or use a different method.
Frames from video RC8_220122_1of2  already extracted (more will be added)!
Loading video...
Duration of video [s]:  247.83188792528352 , recorded @  29.98 fps!
Overall # of frames:  7430 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 247.83  seconds.
Extracting and downsampling... 805  frames from the video.


805it [05:18,  2.53it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [29, 696, 813, 744, 5599, 5673, 3682, 889, 792, 4726, 5680, 5429, 6774, 796, 766, 800, 27, 2866, 492, 6782]
Attempting to create a symbolic link of the video ...
Symlink creation impossible (exFat architecture?): cutting/pasting the video instead.
/content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos/RC8_220122_1of2.mp4 moved to /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos/RC8_220122_1of2.mp4
New videos were added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\RC8_220122_1of2.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.
Method  jump  found  247  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider chang

...

291it [01:57,  2.47it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [3383, 2138, 3978, 4500, 2334, 2767, 2367, 3083, 2693, 3540, 4703, 3476, 25, 1985, 2296, 4363, 3453, 2348, 1224, 2337]
Attempting to create a symbolic link of the video ...
Symlink creation impossible (exFat architecture?): cutting/pasting the video instead.
/content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos/RC7_220126_1of2.mp4 moved to /content/drive/My Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/videos/RC7_220126_1of2.mp4
New videos were added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\RC7_220126_1of2.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.
Method  jump 

/usr/local/lib/python3.7/dist-packages/deeplabcut/refine_training_dataset/outlier_frames.py:390: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  sum_ = temp_dt.sum(axis=1, level=1)


 found  360  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the parameters (start, stop, p_bound, comparisonbodyparts) or use a different method.
Frames from video MsOpto 25 06.11.2021 (1of2)  already extracted (more will be added)!
Loading video...
Duration of video [s]:  154.4362908605737 , recorded @  29.98 fps!
Overall # of frames:  4630 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 154.44  seconds.
Extracting and downsampling... 360  frames from the video.


359it [02:21,  2.26it/s]

**That was step 1. Now. . .**
Now you will need to go back to a computer with GUI capability (i.e., not Colab; e.g., Skywalker or your laptop) and 'refine_labels'. This is done with the GUI. Specifically-


*   Open the GUI
*   Select the config file for this project and load the project
*   Click on the last tab
*   Click LAUNCH
*   Click on 'Load labels' (button bottom left of pop-up)
*   Choose a folder in the 'labeled_videos' folder, select the 'machinelabels*.h5' file.
*   Continue like you were labeling the videos the first time. There will already be labels placed by the network. Most of the time one or more will be wrong. Your job is to fix them. Be sure to hit save (often and) when you're done.




In [ ]:
deeplabcut.merge_datasets(path_config_file)

In [ ]:
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

If you want to pick up training from where you left off before - 
*    Find the train subdirectory within the dlc-models directory and look for the latest snapshot. Open the pose_cfg.yaml file within the same folder and edit the parameter init_weights: '-snapshot-10000' without any filetype ending
*   It probably previously said /Users/ehren/opt/anaconda3/envs/DEEPLABCUT_M1/lib/python3.7/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1,displayiters=100,saveiters=2000)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3', 'objectA'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'clahe': True,
 'claheratio': 0.1,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_all_combined_trackerJun22/all_combined_tracker_jic95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'edge': False,
 'emboss': {'alpha': [0.0, 1.0], 'embossratio': 0.1, 'strength': [0.5, 1.5]},
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'histeq': True,
 'histeqratio': 0.1,
 'init_weights': '/content/drive/My '
                 'Drive/Projects/DeepLabCut/all_combined_tracker-aes-2022-06-22/dlc-models/iteration-0/all_combined_trackerJun22-trainset95shuffle1/snapshot-292000',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_

Selecting single-animal trainer
Batch Size is 1
Loading already trained DLC with backbone: resnet_50


ValueError: ignored

# PUT THAT NETWORK TO WORK!!

You did it! Really! If you made it this far, your network should be working nicely. If not, repeat the last two blocks until you are happy with the way it is working. Once you are happy with it, now you can use the following commands to label your videos. . .

If you're wondering if it is working well, try the following lines out. Look at the labeled videos and analyze the csv file coordinates. If the video is accurately labelled and the coordinates in the csv file are stable (i.e., don't jump around) then your network is likely working well! Just watch for when the rat does strange things, the networks struggle most with these. . . 

**Instructions**:


*   Put your videos that you want analyzed into the folder pointed to by 'testVideo_path'. This is likely the folder called 'videos' in your project folder. 
*   Call the following two commands. Technically, only the first is required. This creates the csv file that matlab will use to analyze the data. The second, however, is useful if you want to preview how well the network labeled the video: it will re-create the video with little dots superimposed on it showing where the network thought each body part was.

*   These commands will put their output files (the csv and labeled video) into the same folder as where you put your videos to be analyzed. 
*    Once each video has been processed, you can remove them from the Google Drive folder and put them back in the appropriate place on the Dropbox file system (e.g., in your project folder or with the original video).

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=False)

In [ ]:
deeplabcut.analyze_videos(path_config_file,testVideo_path, videotype='mp4', save_as_csv=True)
#deeplabcut.create_labeled_video(path_config_file,testVideo_path, videotype='mp4',draw_skeleton=False,save_frames=False)

Using snapshot-1030000 for model /content/drive/My Drive/Projects/DeepLabCut/rear2tracker-jic-2022-01-26/dlc-models/iteration-1/rear2trackerJan26-trainset95shuffle1
Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/My Drive/Projects/DeepLabCut/rear2tracker-jic-2022-01-26/videos/Rear2 10.11.2021 (1of2) Trial-Stim.mp4
Loading  /content/drive/My Drive/Projects/DeepLabCut/rear2tracker-jic-2022-01-26/videos/Rear2 10.11.2021 (1of2) Trial-Stim.mp4
Duration of video [s]:  266.0 , recorded with  29.98 fps!
Overall # of frames:  7974  found with (before cropping) frame dimensions:  1920 1080
Starting to extract posture


 99%|█████████▉| 7900/7974 [12:03<00:06, 10.92it/s]


Saving results in /content/drive/My Drive/Projects/DeepLabCut/rear2tracker-jic-2022-01-26/videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_rear2trackerJan26shuffle1_1030000'

In [ ]:
#Analyzing frames from new animals, produces machine labels
#Change "FolderName" to specific Project Folder if analyzing videos from multiple projects at once

FolderName = 'msopto22tracker-jic-2022-02-17'

new_videos = ['/content/drive/My Drive/Projects/DeepLabCut/'+FolderName+'/videos/MS opto 22 07.30.2021 retrain (1 of 2)', '/content/drive/My Drive/Projects/DeepLabCut/'+FolderName+'/videos/MsOpto 22 02.08.2021 (1of2)', 
              '/content/drive/My Drive/Projects/DeepLabCut/'+FolderName+'/videos/MsOpto 22 03.11.2021 (1of2)', '/content/drive/My Drive/Projects/DeepLabCut/'+FolderName+'/videos/MsOpto 22 06.14.2021 (1of2)']

deeplabcut.analyze_videos(path_config_file,new_videos, videotype='mp4', save_as_csv=True)

In [ ]:
deeplabcut.create_labeled_video(path_config_file,new_videos, videotype='mp4',draw_skeleton=False,save_frames=False)